## Load necessary libraries & datasets

### Libraries & dataset

In [ ]:
import nltk
import re
import regex
import os
from functools import reduce

nltk.download('punkt')
os.chdir("/content/")
!git clone https://github.com/nmng108/ComOM.git

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Cloning into 'ComOM'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 153 (delta 59), reused 93 (delta 8), pack-reused 0
Receiving objects: 100% (153/153), 14.92 MiB | 2.70 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [ ]:
import pandas as pd
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
# !pip install transformers
# !git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git
# !pip install -e "/content/transformers"
# !pip install underthesea

In [ ]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.1 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4307 sha256=4dde0225a5e6de7d7b6a6bdcba829309f64fb64343fda697e8747f26aa4a5347
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
import py_vncorenlp

# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
try:
  os.mkdir('/content/vncorenlp')
  py_vncorenlp.download_model(save_dir='/content/vncorenlp')

  # Load the word and sentence segmentation component
  rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/vncorenlp')
except FileExistsError as e:
  print("The files have been downloaded.")

### Load the Vietnamese stopword list

In [ ]:
abs_path: str = "/content/ComOM/vietnamese-stopwords.txt"

if not os.path.isfile(abs_path):
  print(f"Cannot load set of stopwords as the file \"{abs_path}\" is not found")
  exit(-1)

try:
  with open(abs_path, mode="r") as file:
    global stopwords
    stopwords = file.read().split("\n")
    stopwords = set([w.lower() for w in stopwords])

except Exception as e:
  print(f"Error raised while reading {abs_path}: {str(e)}")

print(f"Loaded {len(stopwords)} stopwords in \"{abs_path}\"")

Loaded 1942 stopwords in "/content/ComOM/vietnamese-stopwords.txt"


### Helper functions

In [ ]:
# @title
def flatten_train_dataset(collapsed_dataset: pd.DataFrame | list[dict]) -> pd.DataFrame:
  """
  Used for dataset with multi-quintuple sentences.
  Convert a n-quintuple sentence into n data points, where each data point corresponds
  to the original sentence and 1 quintuple of it.
  """
  if isinstance(collapsed_dataset, pd.DataFrame):
    assert "sentence" in collapsed_dataset.columns and "quintuples" in collapsed_dataset.columns, (
        "Either sentence or quintuples (or both) not found."
    )

    result: list[dict] = []

    for i, sent_data in collapsed_dataset.iterrows():
      dp = {"sentence": sent_data["sentence"]}

      if isinstance(sent_data["quintuples"], str):
        quins = eval(sent_data["quintuples"])

        if isinstance(quins, list) and len(quins) > 0:
          for quin in quins:
            dp_1 = dp.copy()
            dp_1.update(quin)
            result.append(dp_1)

      else: result.append(dp)

    return pd.DataFrame(result)

  elif isinstance(collapsed_dataset, list)and all([isinstance(d, list) for d in collapsed_dataset]):
    result: list[dict] = []

    for sent_data in collapsed_dataset:
      dp = {"sentence": sent_data.get("sentence")}

      if isinstance(sent_data["quintuples"], str):
        quins = eval(sent_data["quintuples"])

        if isinstance(quins, list) and len(quins) > 0:
          for quin in quins:
            dp_1 = dp.copy()
            dp_1.update(quin)
            result.append(dp_1)

      else: result.append(dp)

    return pd.DataFrame(result)

  else: raise ValueError("Invalid input")

### Load datasets

In [ ]:
# @title
## Initialize DataFrames ##
# Make sure that you have pulled the git repository (first step)
TRAIN_DF: pd.DataFrame = pd.read_csv("/content/ComOM/compilations/post_analysis_training_dataset.csv")
DEV_DF: pd.DataFrame = pd.read_csv("/content/ComOM/compilations/post_analysis_development_dataset.csv")
TEST_DF: pd.DataFrame = pd.read_csv("/content/ComOM/compilations/test_dataset.csv")

print(f"{len(TRAIN_DF)} - {len(DEV_DF)} - {len(TEST_DF)}")

2716 - 1165 - 1732


In [ ]:
## Constants ##

## Flatten training dataset ##
# Should only use this to create DataFrame or replicate this if needed
FLAT_TRAIN_DF = flatten_train_dataset(TRAIN_DF)
FLAT_DEV_DF = flatten_train_dataset(DEV_DF)
####

TOTAL_TRAIN_SENTENCES: int = len(TRAIN_DF)
TOTAL_TEST_SENTENCES: int = len(TEST_DF)

# Training dataset properties
TOTAL_TRAIN_SENTENCES: int = len(TRAIN_DF)
TOTAL_TRAIN_SENTENCES_WITHOUT_QUINTUPLE: int = len(TRAIN_DF[TRAIN_DF.quintuples.isna()])
TOTAL_TRAIN_SENTENCES_CONTAIN_QUINTUPLE: int = TOTAL_TRAIN_SENTENCES - TOTAL_TRAIN_SENTENCES_WITHOUT_QUINTUPLE
TOTAL_TRAIN_QUINTUPLES: int = len(FLAT_TRAIN_DF[FLAT_TRAIN_DF.label.str.contains("^.{1,}$", na=False)])

# Development dataset properties
TOTAL_DEV_SENTENCES: int = len(DEV_DF)
TOTAL_DEV_SENTENCES_WITHOUT_QUINTUPLE: int = len(DEV_DF[DEV_DF.quintuples.isna()])
TOTAL_DEV_SENTENCES_CONTAIN_QUINTUPLE: int = TOTAL_DEV_SENTENCES - TOTAL_DEV_SENTENCES_WITHOUT_QUINTUPLE
TOTAL_DEV_QUINTUPLES: int = len(FLAT_DEV_DF[FLAT_DEV_DF.label.str.contains("^.{1,}$", na=False)])

# Test dataset property
TOTAL_TEST_SENTENCES: int = len(TEST_DF)

####

In [ ]:
TRAIN_DF.quintuples.tolist()[1] == np.nan

False

In [ ]:
# Test constants & variables created above
print(f"""Total number of sentences (followed by 'quin-owners/non-quin'):
1. Training dataset: {TOTAL_TRAIN_SENTENCES} ({TOTAL_TRAIN_SENTENCES_CONTAIN_QUINTUPLE} / {TOTAL_TRAIN_SENTENCES_WITHOUT_QUINTUPLE})
  (After flattening dataset based on quintuple list)
  - Total number of data points: {len(FLAT_DEV_DF)}
  - Total number of quintuples (or number of data points containing quintuple): {TOTAL_TRAIN_QUINTUPLES}
2. Development dataset: {TOTAL_DEV_SENTENCES} ({TOTAL_DEV_SENTENCES_CONTAIN_QUINTUPLE} / {TOTAL_DEV_SENTENCES_WITHOUT_QUINTUPLE})
  (After flattening dataset based on quintuple list)
  - Total number of data points: {len(FLAT_DEV_DF)}
  - Total number of quintuples (or number of data points containing quintuple): {TOTAL_DEV_QUINTUPLES}
3. Test dataset: {TOTAL_TEST_SENTENCES}
""")

Total number of sentences (followed by 'quin-owners/non-quin'):
1. Training dataset: 2716 (812 / 1904)
  (After flattening dataset based on quintuple list)
  - Total number of data points: 1287
  - Total number of quintuples (or number of data points containing quintuple): 1089
2. Development dataset: 1165 (349 / 816)
  (After flattening dataset based on quintuple list)
  - Total number of data points: 1287
  - Total number of quintuples (or number of data points containing quintuple): 471
3. Test dataset: 1732



## Data cleaning

Character candidates to be removed: `^ -` ` ?: ?` `( )` `,` `.` (sent-separator and terminator) `. . .` (tripdots) ` ?' ?` ` ?" ?` `!` `&` ``

In [ ]:
train_df = TRAIN_DF.copy()

In [ ]:
# stringify quins

# tmp_df = pd.DataFrame([{
#     'index': i,
#     'sentence': r.sentence,
#     'quintuples': str(r.quintuples)
#     } for i, r in train_df.iterrows() if r.quintuples is not None])
# tmp_df.set_index("index")
tmp_df = train_df.copy()
tmp_df.quintuples = train_df.quintuples.astype(str)
tmp_df.query("quintuples.str.contains('^\[') & quintuples.duplicated(keep=False)")

In [ ]:
# Check substring
sentences = train_df.sentence.unique()

for i, s in enumerate(sentences):
  if len(nltk.word_tokenize(s)) < 5: continue
  wrappers = train_df.loc[[i for i, sent in train_df.sentence.items() if sent.lower().find(s.lower()) > -1 and not sent.lower() == s.lower()]]
  wrap_strs = wrappers[wrappers.quintuples.notna()].sentence.unique().tolist()
  if len(wrap_strs) > 0:
    print(f"Tuple {i}: with the sentence '{s}'\n{wrap_strs}")

In [ ]:
# Remove duplicates
# especially this case (the third one contains terminating '.')
tmp_df = pd.DataFrame([
    {"index":992,"sentence":"alt : Cả hai dòng máy đều có tiện ích nổi bật của hãng","quintuples":None},
    {"index":993,"sentence":"des : Cả hai dòng máy đều có tiện ích nổi bật của hãng","quintuples":"{'subject': 'Cả hai dòng máy', 'object': 'Cả hai dòng máy', 'aspect': 'tiện ích nổi bật của hãng', 'predicate': 'đều có', 'label': 'EQL'}"},
    {"index":994,"sentence":"Cả hai dòng máy đều có tiện ích nổi bật của hãng .","quintuples":"{'subject': 'Cả hai dòng máy', 'object': 'Cả hai dòng máy', 'aspect': 'tiện ích nổi bật của hãng', 'predicate': 'đều có', 'label': 'EQL'}"}
    ])
# remove_duplicates(tmp_df, mode="non-quin")#.loc[992:994]

# print(f"\nSize of training set after modified: {len(train_df)}")

,index,sentence,quintuples
1,993,des : Cả hai dòng máy đều có tiện ích nổi bật ...,"{'subject': 'Cả hai dòng máy', 'object': 'Cả h..."
2,994,Cả hai dòng máy đều có tiện ích nổi bật của hãng,"{'subject': 'Cả hai dòng máy', 'object': 'Cả h..."


### Utility functions

In [ ]:
def make_BoW(str_list: list[str], desc=True):
  """ Create a Bag of Word """
  bow: dict = {}

  for subject in str_list:
    tokens = nltk.word_tokenize(subject)

    for t in tokens:
      t = t.lower()
      if bow.get(t) is None: bow.update({t: 1})
      else: bow.update({t: bow.get(t) + 1})

  return dict(sorted(bow.items(), key=lambda w: w[1], reverse=desc))

In [ ]:
def render_wordcloud(values: str | list[str], stopwords: list = None,
                     width: int = 500, height: int = 500,
                     max_font_size: int = 50, min_font_size: int = 5,
                     max_words: int = 350, background_color = "white"):
  is_str_list: bool = isinstance(values, list) and all([isinstance(v, str) for v in values])

  assert is_str_list or isinstance(values, str), (
      "values must be a string or a list of strings"
  )

  text: str = " ".join([s for s in values]) if is_str_list else values
  wordcloud = WordCloud(stopwords=stopwords,
                        width=width, height=height, # image resolution
                        min_font_size=min_font_size, max_font_size=max_font_size,
                        max_words=max_words, background_color=background_color
                        ).generate(text)

  plt.figure(figsize=[8,8]) # scale the image
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  plt.show()

### `remove_non_word(df: DataFrame): list` *(TODO: modify this)*

- **Input**:

- **Output**:


In [ ]:
from multipledispatch import dispatch

# @dispatch(list)
def remove_non_word(df: pd.DataFrame):
  pass


# @dispatch(str)
# def parse_and_clean_data(dataset: str = ""):
#   # split by "\n" to sequentially read each line of the input
#   dataset_array: list = dataset.split("\n")

#   return parse_and_clean_data(dataset_array)

In [ ]:
filtered_signs = "( -)|( ?: ?)|\(|\)|\,|( \. ?)|( ?\. )|(\.( \.){1,})|'|\"|&|\!"
flattened_train_df[flattened_train_df.predicate.str.contains(filtered_signs, na=False)]

### Correcting data

In [ ]:
selected_sent = train_df.iloc[805:807]
selected_sent

,sentence,quintuples
805,Mặc dù dải động của cảm biến trên iPhone SEmớ...,"[{'subject': 'iPhone SEmới', 'object': 'iPhon..."
806,Điện thoại iPhone 13 Pro cũng có cảm biến hì...,"[{'subject': 'Điện thoại iPhone 13 Pro', 'ob..."


In [ ]:
# Add 1 more quintuple to dev set
new_row = {
    "sentence": "GPU cũng chỉ đến thế khi nó vẫn đứng ở cuối bảng so sánh của chúng ta .",
    "subject": "GPU",
    "predicate": "đứng ở cuối",
    "label": "SUP-"
}
flat_dev_df = FLAT_DEV_DF.append(new_row, ignore_index=True)
flat_dev_df[flat_dev_df.sentence.str.contains("đến thế")].query("label.str.contains('SUP-', na=False)")

<ipython-input-91-34419329a384>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  FLAT_DEV_DF.append(new_row, ignore_index=True).query("label.str.contains('SUP-', na=False)")


,sentence,subject,object,aspect,predicate,label
315,GPU cũng chỉ đến thế khi nó vẫn đứng ở cuối bả...,nó,,,đứng ở cuối,SUP-
1287,GPU cũng chỉ đến thế khi nó vẫn đứng ở cuối bả...,GPU,NaN,NaN,đứng ở cuối,SUP-


In [ ]:
FLAT_TRAIN_DF[FLAT_TRAIN_DF.sentence.str.contains("hiển thị")].query("subject == object")

,sentence,subject,object,aspect,predicate,label
680,"Về màn hình , cả hai cùng có tấm nền Super AMO...",cả hai,cả hai,màn hình,cùng có tấm nền Super AMOLED Full HD +,EQL
681,"Về màn hình , cả hai cùng có tấm nền Super AMO...",cả hai,cả hai,màn hình,kích thước chênh lệch rất ít,EQL
682,"Về màn hình , cả hai cùng có tấm nền Super AMO...",,,chất lượng hiển thị,không khác nhau nhiều,EQL
1226,Điều đó cho phép không gian hiển thị lớn hơn v...,,,không gian hiển thị,lớn hơn,COM+
1740,Màn hình lớn sẽ giúp bạn có không gian giải tr...,cả hai,cả hai,tấm nền cao cấp,đều được trang bị,EQL
2169,Một màn hình IPS LCD hiển thị yếu hơn nhưng lạ...,,,màn hình,hiển thị yếu hơn,COM-
2569,Không có sự thay đổi lớn về kích thước hiển th...,2 dòng điện thoại này,2 dòng điện thoại này,kích thước hiển thị màn hình,Không có sự thay đổi lớn,EQL
2570,"Về chất lượng hiển thị , so sánh iPhone XS và ...",iPhone XS,iPhone XS,chất lượng hiển thị,có phần trội hơn,COM+


## Post-cleaned data analysis

In [ ]:
# Check if any sentence is duplicated
# origin_idx: key, value(origin_sent, relationship, sub_str_idx, sub_str_sent)
result: list = []

for i in range(len(training_dataset)):
  current_sentence: str = training_dataset[i].sentence.lower()

  for j in range(i - 1, -1, -1):
    prev_sentence: str = training_dataset[j].sentence.lower()
    rel = None

    if prev_sentence == current_sentence: rel = "equal"
    elif prev_sentence in current_sentence: rel = "contains"
    elif current_sentence in prev_sentence: rel = "contained"

    if not rel is None: result.append({
        "origin_idx": i,
        "origin_sent": training_dataset[i].sentence,
        "rel": rel,
        "2nd_sent_idx": j,
        "2nd_sent": training_dataset[j].sentence
    })

pd.DataFrame(result)

,origin_idx,origin_sent,rel,2nd_sent_idx,2nd_sent
0,6,"Trong ba smartphone ngày hôm nay, Motorola One...",contains,3,Thiết kế
1,7,Nó không sử dụng thiết kế notch hình đục lỗ mà...,contains,3,Thiết kế
2,11,Màn hình,contained,6,"Trong ba smartphone ngày hôm nay, Motorola One..."
3,15,Chính vì vậy mà di động Samsung chiến thắng tr...,contains,11,Màn hình
4,16,"Realme 6 Pro có màn hình IPS, nhưng nó có tốc ...",contains,11,Màn hình
...,...,...,...,...,...
158925,4166,des: Yên tâm mua hàng tại Thế Giới Di Động,contains,53,des:
158926,4167,Yên tâm mua hàng tại Thế Giới Di Động.,contains,1286,Thế giới di động.
158927,4168,Lưu ý : Bạn có thể cập nhật chi tiết chính sác...,equal,2707,LƯU Ý : Bạn có thể cập nhật chi tiết chính sác...
158928,4170,Hẹn gặp lại bạn ở các bài viết sau!,equal,3343,Hẹn gặp lại bạn ở các bài viết sau!


In [ ]:
from tensorflow import keras
from typing import List
from keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too (likes movies)."]

def print_bow(sentence: List[str]) -> None:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentence)
    sequences = tokenizer.texts_to_sequences(sentence)
    word_index = tokenizer.word_index

    print(f"Vocab: {word_index}")
    print(sequences)

    bow = {}
    for key in word_index:
        bow[key] = sequences[0].count(word_index[key])

    print(f"Bag of word for sentence 1:\n{bow}")
    print(f"We found {len(word_index)} unique tokens.")

print_bow(sentence)

{'likes': 1, 'movies': 2, 'john': 3, 'to': 4, 'watch': 5, 'mary': 6, 'too': 7}
[[3, 1, 4, 5, 2, 6, 1, 2, 7, 1, 2]]
Bag of word sentence 1:
{'likes': 3, 'movies': 3, 'john': 1, 'to': 1, 'watch': 1, 'mary': 1, 'too': 1}
We found 7 unique tokens.


# Word Segmentation & Tokenization

libs: VNCoreNLP, PyVi, PhoBert

In [ ]:
# Test
text = "title: So sánh Galaxy Z Fold 4 và iPhone 13 Pro Max: Flagship nào hấp dẫn hơn?	title : So sánh Galaxy Z Fold 4 và iPhone 13 Pro Max : Flagship nào hấp dẫn hơn ?"
rdrsegmenter.word_segment(text)

['title : So sánh Galaxy_Z_Fold 4 và iPhone 13 Pro_Max : Flagship nào hấp dẫn hơn ? title : So sánh Galaxy_Z_Fold 4 và iPhone 13 Pro_Max : Flagship nào hấp dẫn hơn ?']

In [ ]:
flat_df = FLAT_TRAIN_DF.copy()
check_identical = []

for i, d in flat_df.iterrows():
  sent = " ".join(rdrsegmenter.word_segment(d.sentence))
  d.sentence = sent
  is_identical = True

  if isinstance(d.subject, str) and len(d.subject) > 1:
    s = rdrsegmenter.word_segment(d.subject)[0]
    d.subject = s

    if sent.find(s) == -1:
      is_identical = False
      pattern = regex.sub("[_ ]", "[_ ]", s)
      sent_substr = regex.findall(pattern, sent)[0]

      if len(regex.findall("_", sent_substr)) < len(regex.findall("_", s)):
        d.sentence = regex.sub(pattern, s, sent)
      else: d.subject = sent_substr

  if isinstance(d.object, str) and len(d.object) > 1:
    s = rdrsegmenter.word_segment(d.object)[0]
    d.object = s
    if sent.find(s) == -1:
      is_identical = False
      pattern = regex.sub("[_ ]", "[_ ]", s)
      sent_substr = regex.findall(pattern, sent)[0]

      if len(regex.findall("_", sent_substr)) < len(regex.findall("_", s)):
        d.sentence = regex.sub(pattern, s, sent)
      else: d.object = sent_substr

  if isinstance(d.aspect, str) and len(d.aspect) > 1:
    s = rdrsegmenter.word_segment(d.aspect)[0]
    d.aspect = s
    if sent.find(s) == -1:
      is_identical = False
      pattern = regex.sub("[_ ]", "[_ ]", s)
      sent_substr = regex.findall(pattern, sent)[0]

      if len(regex.findall("_", sent_substr)) < len(regex.findall("_", s)):
        d.sentence = regex.sub(pattern, s, sent)
      else: d.aspect = sent_substr

  if isinstance(d.predicate, str) and len(d.predicate) > 1:
    s = rdrsegmenter.word_segment(d.predicate)[0]
    d.predicate = s
    if sent.find(s) == -1:
      is_identical = False
      pattern = regex.sub("[_ ]", "[_ ]", s)
      sent_substr = regex.findall(pattern, sent)[0]

      if len(regex.findall("_", sent_substr)) < len(regex.findall("_", s)):
        d.sentence = regex.sub(pattern, s, sent)
      else: d.predicate = sent_substr

  check_identical.append(is_identical)

flat_df["identical_segments"] = check_identical

In [ ]:
FLAT_TRAIN_DF.query("label.isna()")

In [ ]:
# Check if the segmentation stage worked correctly
flat_df[flat_df.identical_segments == False]

,sentence,subject,object,aspect,predicate,label,identical_segments
109,Đây có_lẽ là nơi cho thấy sự khác_biệt rõ_ràng...,một chiếc máy cao_cấp,một chiếc điện_thoại tầm_trung,,sự khác_biệt rõ_ràng nhất,DIF,False
285,"Mẫu điện_thoại có thiết_kế vuông sang_trọng , ...",Mẫu điện_thoại,những chiếc smartphone cao_cấp trên thị_trường,thiết_kế vuông,"sang_trọng , tinh_tế không khác_gì",EQL,False
2516,Và S23 Ultra dường_như có tất_cả các bộ_phận t...,S23 Ultra,X90 Pro,bộ_phận tạo nên hệ_thống hình_ảnh đồng_bộ_hoá,tốt hơn,COM+,False
2596,Camera được tăng số megapixel trên camera self...,Camera,,khẩu_độ,được giữ_nguyên,EQL,False
2710,"Tương_tự , thì hệ_thống camera vẫn giữ_nguyên ...",hệ_thống camera,,thiết_kế,giữ_nguyên,EQL,False
2896,Xét về pin giữa A23 và A32 về dung_lượng pin t...,cả hai,,dung_lượng pin,cao nhất được Samsung trang_bị cho những dòng ...,SUP+,False
2979,Còn trong khi đó Galaxy S10 Plus có dải tương_...,Galaxy S10 Plus,,dải tương_phản_động,tốt hơn,COM+,False
